# load images

In [ ]:
import os 
import shutil
import imageio
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from ipywidgets import interact 


subj = 2
time = 9

img = np.load('/Users/wangheehwan/Desktop/simmim_img_seq20_timepatch1_masking0.8.npy')
pred = np.load('/Users/wangheehwan/Desktop/simmim_pred_seq20_timepatch1_masking0.8.npy')
mask = np.load('/Users/wangheehwan/Desktop/simmim_mask_seq20_timepatch1_masking0.8.npy')

img_subj = img[subj]
pred_subj = pred[subj]
mask_subj = 1 - mask[subj]
diff_subj = np.abs(img_subj - pred_subj) 
masked_diff_subj = mask_subj * diff_subj

s_max = img.shape[2]
t_max = img.shape[-1]
vmin = np.min(diff_subj)
vmax = np.max(diff_subj)





# visualize spatial axis

In [ ]:
def sagittal_view(x,): 
    plt.figure()
    #plt.imshow(img_subj[0,x,:,:,time], cmap='gray')
    plt.imshow(pred_subj[0,x,:,:,time], cmap='Reds')
    #plt.imshow(diff_subj[0,x,:,:,time], cmap='Reds', vmin=vmin, vmax=vmax)
    #plt.imshow(masked_diff_subj[0,x,:,:,time], cmap='Reds', vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.show()

def coronal_view(x,): 
    plt.figure()
    #plt.imshow(img_subj[0,:,:,x,time], cmap='gray')
    plt.imshow(pred_subj[0,:,:,x,time], cmap='Reds')
    #plt.imshow(diff_subj[0,:,:,x,time], cmap='Reds', vmin=vmin, vmax=vmax)
    #plt.imshow(masked_diff_subj[0,x,:,:,time], cmap='Reds', vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.show()

def horizontal_view(x,): 
    plt.figure()
    #plt.imshow(img_subj[0,:,x,:,time], cmap='gray')
    plt.imshow(pred_subj[0,:,x,:,time], cmap='Reds')
    #plt.imshow(diff_subj[0,:,x,:,time], cmap='Reds', vmin=vmin, vmax=vmax)
    #plt.imshow(masked_diff_subj[0,x,:,:,time], cmap='Reds', vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.show()
print(f'Loss: {np.mean(diff_subj)}')

interact(sagittal_view, x=(0,s_max-1))
interact(coronal_view, x=(0,s_max-1))
interact(horizontal_view, x=(0,s_max-1))

# visualize temporal axis

In [ ]:
slice_num = 48

def temporal_view(x,):
    plt.figure()
    #plt.imshow(img_subj[0,:,:,slice_num,x], cmap='gray')
    plt.imshow(pred_subj[0,:,:,slice_num,x], cmap='Reds')
    #plt.imshow(diff_subj[0,:,:,slice_num,x], cmap='Reds', vmin=vmin, vmax=vmax)
    #plt.imshow(masked_diff_subj[0,:,:,slice_num,x], cmap='Reds', vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.show()

interact(temporal_view, x=(0,t_max-1))

# visualize temporal dynamics of specific coordinates 

In [ ]:
coord_x, coord_y, coord_z = 70, 45, 48

labels =  [f'gt (mean={np.mean(img_subj[0,coord_x,coord_y,coord_z,:])})', f'pred (mean={np.mean(pred_subj[0,coord_x,coord_y,coord_z,:])})', f'diff (mean={np.mean(diff_subj[0,coord_x,coord_y,coord_z,:])})']


def temporal_view(x,):
    plt.figure()
    plt.imshow(img_subj[0,:,:,coord_z,x], cmap='gray')
    plt.plot([coord_x], [coord_y], 'bo')
    #plt.imshow(pred_subj[0,:,:,coord_z,x], cmap='Reds')
    #plt.imshow(diff_subj[0,:,:,coord_z,x], cmap='Reds', vmin=vmin, vmax=vmax)
    #plt.imshow(masked_diff_subj[0,:,:,coord_z,x], cmap='Reds', vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.show()
interact(temporal_view, x=(0,t_max-1))

plt.close()
plt.figure()
plt.plot(img_subj[0,coord_x,coord_y,coord_z,:], label=labels[0])
plt.plot(pred_subj[0,coord_x,coord_y,coord_z,:], label=labels[1])
#plt.plot(diff_subj[0,x,y,z,:], label=labels[2])
plt.legend(loc='upper right')
plt.show


# make gif file

In [ ]:
tmp_work_dir = '/Users/wangheehwan/Desktop/tmp_gif'
gif_file = '/Users/wangheehwan/Desktop/simmim_pred_seq20_timepatch1_masking0.8.gif'

# make temporal working directory
if os.path.isdir(tmp_work_dir) is False:
    os.mkdir(tmp_work_dir) 

# stack 2D slices 
gif_np = [pred_subj[0,:,:,48,x] for x in range(t_max)]

# save array as plot png
file_names = []
for i in range(len(gif_np)): 
    file_name = os.path.join(tmp_work_dir,f'simmim_pred_seq20_timepatch1_masking0.8_t{i}.png')
    file_names.append(file_name)
    plt.imshow(pred_subj[0,:,:,48,i], cmap='Reds')
    plt.savefig(file_name)
    plt.close()

# load plot png
frames = []
for file_name in file_names:
    if file_name.endswith(".png"):
        frames.append(imageio.imread(file_name))

# make gif
imageio.mimsave(gif_file, frames, format='GIF', duration=1)

# remove intermediate files (plot png)
for file_name in set(file_names):
    os.remove(file_name)

# remove temporal working directory
if os.path.isdir(tmp_work_dir) is False:
    shutil.rmtree(tmp_work_dir)